# ENGR418 Project Stage 2 Group 31

By: Jared Paull (63586572), Liam Ross (75469692)


In [216]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import confusion_matrix
from PIL import Image, ImageFilter
import PIL
from sklearn.neighbors import KNeighborsClassifier
import sklearn

## Single Function Call

The function in the cell below can be called to run the entire algorithm. Before running, be sure to run the cells containing the functions at the bottom of this page, or else errors will be thrown.

In [223]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import confusion_matrix
from PIL import Image, ImageFilter
import PIL
from sklearn.neighbors import KNeighborsClassifier
import sklearn

# first param is the relative training data directory
# second param is the relative testing data directory

training_data_relative_dir = "../data/training"
testing_data_relative_dir = "../data/testing"

# this will take a 1-2 minutes to run (depending on device capabilities)
test_function(training_data_relative_dir, testing_data_relative_dir)

Shape Predicted  Circle  Rectangle  Square
Shape Actual                              
Circle               27          0       0
Rectangle             0         27       0
Square                0          0      27
Percentage of correct classification from model on training data set: 100.00%

Shape Predicted  Circle  Rectangle  Square
Shape Actual                              
Circle               24          0       3
Rectangle             0         27       0
Square                0          0      27
Percentage of correct classification from model on testing data set: 96.30%


## Setting Tuning Parameters

pefore starting, these parameters must be set, they can be tuned to optimize performance though. The optimal values found are considered as default below

In [238]:
# image_size=64, filter_value=4, angles=0->180 increments by 2, 100% & 96.30%

# image size will dictate the size each image will be reshapet to later, used for tuning
image_size = 64
# filter value sets a threshold value on the edge detection image, used for tuning
filter_value = 4
# list of angles that the algorithm will rotate though, used for tuning
angles = []
for i in range(90):
    angles.append(2*i)

### Feature Engineering

Next, all image data is scrapped from the image files in their respective relative directory. Refer to get_image_feature_data function for line by line description. In essense, each image will have 4 feature vectors. One for maximum Lego brick length, one for minimum lego brick length, and one for both average and median Lego brick length. These values will come from the rotated image, which is further discussed in their own respective functions

In [239]:
# gets all training data from relative directory.
# refer to functions at bottom for line-by line commenting
x,y = get_image_feature_data("../data/training", image_size, filter_value, angles)
# gets all testing data from relative directory.
xt, yt = get_image_feature_data("../data/testing", image_size, filter_value, angles)

### Classifier Training

Now with the feature vectors selected and image data collected, a classifier can be trained. After testing, we opted to use a k neighbors classifier which provided the best results. The classifier will classify a point based on the classification of points near it, making it simple and quick to calcualte. With strong engineered features, the k neighbors classifier provides a high accuracy solution.

In [240]:
# creates a non linear k-nearest neighbor classifier that considers the 8th nearest neighbors where each neighbor is weighted by distance
from sklearn.neighbors import KNeighborsClassifier
k_neighbors = KNeighborsClassifier(n_neighbors=8, weights="distance")

# fits data to the classifier
k_neighbors.fit(x,y);

## **Prediction, Confusion Matrices, and Accuracy of Classifier**

### Classifier Prediction: Training Data

Now, the classifier can be tested. First it is tested with the training image data. The results of the confusion matrix, as well as the accuracy of the algorithm are shown below.

In [241]:
# feeds the training data back into the classifier for predicition
pred =  k_neighbors.predict(x)
# formats the prediction values to string labels (refer to function below)
predicted = confusion_format(pred)
# foramts the actual labels to string labels (refer to function below)
actual = confusion_format(y)
# prints the confusion matrix using string labels
print(pd.crosstab(actual, predicted, rownames=["Shape Actual"], colnames=["Shape Predicted"]))
# prints the error percentage (refer to function below)
print(f"Percentage of correct classification from model on training data set: {100-error_percentage(pred,y):.2f}%\n")

Shape Predicted  Circle  Rectangle  Square
Shape Actual                              
Circle               27          0       0
Rectangle             0         27       0
Square                0          0      27
Percentage of correct classification from model on training data set: 100.00%



### Classifier Prediction: Testing Data

Next it is tested with the testing image data. The testing image is a better recognition of the classifiers accuracy since it is being fed images that it has never seen before. The results of the confusion matrix, as well as the accuracy of the algorithm are shown below.

In [242]:
# feeds the testing data into the classifier for prediction
pred =  k_neighbors.predict(xt)
# formats the prediction values to string labels (refer to function below)
predicted = confusion_format(pred)
# formats teh actual labels to string values (refer to function below)
actual = confusion_format(yt)
# prints the confusion matrix using string labels
print(pd.crosstab(actual, predicted, rownames=["Shape Actual"], colnames=["Shape Predicted"]))
# prints the error percentage (refer to function below)
print(f"Percentage of correct classification from model on testing data set: {100-error_percentage(pred,y):.2f}%")

Shape Predicted  Circle  Rectangle  Square
Shape Actual                              
Circle               24          0       3
Rectangle             0         27       0
Square                0          0      27
Percentage of correct classification from model on testing data set: 96.30%


---
---
---
---

# **Functions**

All of these functions **must** be ran before anything else. Each function has its purpose discussed, and are each well commented on.


### edge_image

This function takes in a raw Lego brick image, then exports a filtered, binary, edge detected version. This means, it will output an image that only contains 0/1 in monochrome. All 1's will dictate edges of the Lego brick, while 0's indicate blank space that is not useful. All noise outside of the lego bricks edge should be filtered to allow the get_len to get the correct brick length. Noise going into the get_len function will make the classifier unrealiable. The function is commented on in detail below.

In [4]:
# returns image that is the filtered, reshaped, edge detection version
def edge_image(image, image_size, filter_value):
    # takes input image and converts to monochrome
    image = image.convert("L")
    # converts the monochrome image to an edge detection version (note this image is ripe with noise)
    image = image.filter(ImageFilter.FIND_EDGES)
    # Compress image down to 18x18 image, will blur specific noise in the image to make lego brick obvious
    image = image.resize((16 + 2,16 + 2))
    # simply slices off the outter pixel of the image, border/edge pixels are recognized as ...
    # a "change" in colour, thus are labeled as an edge, the next line will slice out this edge error.
    # will output a 16x16 image
    image = PIL.Image.fromarray(np.array(image)[int(1) : int(image.height -1), int(1) : int(image.width - 1)])
    # resizes image from 16x16 to desizered image size, return information to the plot.
    # resizing down then back up was to blur out and specific noise, so all noise can be easily filtered later.
    image = image.resize((image_size,image_size))
    
    # converts the image to a numpy array
    data = np.asarray(image)
    # filters out any noise in the image
    data[data <= filter_value] = 0
    # converts image from monochrome values to binary (for ease of interpretation)
    data[data > 0] = 1
    
    # converts the image data back to a Pillow image object for further use
    image = PIL.Image.fromarray(data)
    return image

### get_len

used to get the length between the top-most pixel, and the bottom-most pixel in the image. Takes image from edge_image function and will return a single integer representing the lenght described above by rotating slowly, and taking length at each step. We can piece together what the lego brick is by examining the values it takes as it rotates. Function takes image from edge_image function above, thus requires very little background noise to work effectively.

Expect circles to remain similar in value as it rotates. Expect rectangles to have a large maximum value. Expect circles to have a maximum value greater than circle but less then rectangle. Will use max/min/avg/med later to examine the changes over angle

In [243]:
def get_len(image):
    # converts image to numpy array
    data = np.array(image)
    # represents the lowest pixel index (index represents height where bottom of image is zero)
    # initialize quantity to top of image to guarantee it will decrease (assuming image has a non-zero pixel)
    min_index = image.height
    # represents the highest pixel index (index represents height where top of image is maximum value, i.e. image height)
    # initialize quantity to bottom of image to guarantee it will increase (assuming image has a non-zero pixel)
    max_index = 0
    
    # first loop starts from bottom of image and will crawl upwards
    for i in range(image.height):
        # nested loop will examing each pixel from left to right by height
        for j in range(image.width):
            # if a edge is detected (lego brick is found)
            if( data[i][j] == 1):
                # sets min index if current height index is less then smallest index found far
                if (min_index > i):
                    min_index = i
                # sets max index if current height index is greater than greatest index found thus far
                if( max_index < i):
                    max_index = i
    # finally, return difference between max height and min height to get vertical length the image takes up
    return max_index - min_index

### get_image_feature_data

Will iterate through a directory and gather feature data from each image, as well as its corresponding correct label. Is largely an accumulation of edge_image and get_len functions that is iterated for each image. Will return *x*, and *y* which are the feature vectors, and feature labels respectively.

In [6]:
def get_image_feature_data(rel_dir, image_size, filter_value, angles):
    # initializes feature data and labels for use population later
    x = []
    y = []

    # will loop through each file in rel_dir directory
    for pic in os.listdir(rel_dir):
        # creates new Pillow image object from pic in relative directory
        image = PIL.Image.open(f"{rel_dir}/{pic}")
        # calls function to get filtered, reshaped, edge detection version of image.
        image = edge_image(image, image_size, filter_value)
        
        # initialize list to propogate with lengths for different angles
        vec = []
    
        # for each loop to rotate through all angles the algorithm considers
        for angle in angles:
            # rotates original image by angle in for each loop
            img = image.rotate(angle)
            # for specific angle, find the length using the function described above
            length = get_len(img)
            # append new length to list containing length for each angle
            vec.append(length)
        # converts list to array to make math more efficient
        vec = np.array(vec)
        # maximum length recorded between all angles, normalized by height
        # useful for identifying rectangles
        max_len = np.max(vec) / img.height
        # minimum length recorded between all angles, normalized by height
        min_len = np.min(vec) / img.height
        # average length recorded between all angles, normalized by height
        # useful for identifying circles
        avg_len = np.average(vec) / img.height
        # median length recorded between all angles, normalized by height
        # useful for identifying circles
        med_len = np.median(vec) / img.height
        # dynamically override vec to be list of 4 key values from list of lengths by angle
        vec = [max_len, min_len, avg_len, med_len]

        # examine the name of the picture file, can find correct label based on first letter of the file name.
        # c indicates the picture is a circle
        if( str.lower(pic[0]) == "c"):
            # classify circles as a 0
            y.append(0)
        # r indicates the picture is a rectangle
        elif (str.lower(pic[0]) == "r"):
            # classify rectangle as a 1
            y.append(1)
        # only other situation is the image is a square
        else:
            # classify square as a 2
            y.append(2)

        x.append(vec) # each image has 1536 features

    # convert feature vector data/labels from lists to arrays for ease of use in the classifier model
    x = np.array(x)
    y = np.array(y)
    
    return x,y

In [245]:
# This function will convert from decimal label to strings. Very easy to comprehend
# 0=>Circle, 1=>Rectangle, 2=>Square
def confusion_format(labels):
    test = []
    for i in labels:
        if i == 0:
            test.append("Circle")
        elif i == 1:
            test.append("Rectangle")
        else:
            test.append("Square")
    test = np.array(test)
    return test

In [244]:
# Used to calculate error percentage by looking at number of differences between prediction and actual labels.
def error_percentage(pred, y):
    # the number of errors is the number of differences between the model's labels and the correct labels
    errors = 0
    for i in range(pred.size):
        # pred is the predicted array labels, while y is the actual
        if pred[i] != y[i]:
            errors = errors + 1
            
    # then the percentage of errors is the number of errors divided by the total number of image samples times 100 for percentage.
    return errors / pred.size * 100

In [246]:
# Function is exclusively to demonstrate all code in a single function call. Refer to cells above, or functions described for ...
# in depth description/line-by-line description.
def test_function(training_dir, testing_dir):
    image_size = 64
    filter_value = 4
    angles = []
    for i in range(90):
        angles.append(2*i)
    
    x,y = get_image_feature_data(training_dir, image_size, filter_value, angles)
    xt, yt = get_image_feature_data(testing_dir, image_size, filter_value, angles)
    k_neighbors = KNeighborsClassifier(8, weights="distance")
    k_neighbors.fit(x,y);
    
    
    pred =  k_neighbors.predict(x)
    predicted = confusion_format(pred)
    actual = confusion_format(y)
    print(pd.crosstab(actual, predicted, rownames=["Shape Actual"], colnames=["Shape Predicted"]))
    print(f"Percentage of correct classification from model on training data set: {100-error_percentage(pred,y):.2f}%\n")
    
    pred =  k_neighbors.predict(xt)
    predicted = confusion_format(pred)
    actual = confusion_format(yt)
    print(pd.crosstab(actual, predicted, rownames=["Shape Actual"], colnames=["Shape Predicted"]))
    print(f"Percentage of correct classification from model on testing data set: {100-error_percentage(pred,y):.2f}%")